In [11]:
import serial
import tkinter as tk
import time
import cv2
import numpy as np
from ultralytics import YOLO

# Open a serial connection to Arduino
ser = serial.Serial('COM4', 9600, timeout=1)  # Replace 'COM5' with your port name
kamera1=cv2.VideoCapture(0)
kamera2=cv2.VideoCapture(1)

# Create a global variable for the main window
main_window = None

capture_objects = {
    "cam1": cv2.VideoCapture(0),
    "cam2": cv2.VideoCapture(1),
    # Add more camera capture objects if needed
}

def flush_webcams():
    # Release all camera capture objects
    for capture_object in capture_objects.values():
        capture_object.release()

    # Close any opened OpenCV windows
    cv2.destroyAllWindows()

def send_data(input_data):
    time.sleep(1)
    ser.write(input_data.encode())

def read_data():
    while True:
        if ser.in_waiting > 0:
            data = ser.readline().decode().strip()
            output_text.insert(tk.END, data + '\n')
        serial_window.update_idletasks()  # Update the GUI

def close_serial():
    ser.close()
    serial_window.destroy()

def create_serial_window():
    global serial_window

    # Create the serial data window
    serial_window = tk.Toplevel(main_window)
    serial_window.title("Serial Data")
    serial_window.geometry("400x300")

    # Create output text area
    global output_text
    output_text = tk.Text(serial_window, width=40, height=10)
    output_text.pack()

    # Create a close button
    close_button = tk.Button(serial_window, text="Close", command=close_serial)
    close_button.pack()

    # Function to continuously read data from serial
    def read_serial():
        if ser.in_waiting > 0:
            data = ser.readline().decode().strip()
            output_text.insert(tk.END, data + '\n')
        serial_window.after(100, read_serial)  # Schedule the next reading

    # Start reading data from serial
    read_serial()
    
def correction(coordinate):
    def yes():
        send_data("1")
        confirm_window.destroy()
        inputcorrection()

    def no():
        confirm_window.destroy()

    # Create a new window for confirmation
    confirm_window = tk.Toplevel(main_window)
    confirm_window.title("Is there any correction?")

    # Show the coordinate
    coordinate_label = tk.Label(confirm_window, text="Selected Coordinate:")
    coordinate_label.pack()

    coordinate_value = tk.Label(confirm_window, text=f"({coordinate[0]}, {coordinate[1]})")
    coordinate_value.pack()

    coordinate_value = tk.Label(confirm_window, text=f"Is there any correction?")
    coordinate_value.pack()

    # Create buttons for confirmation and cancellation
    confirm_button = tk.Button(confirm_window, text="Yes", command=yes)
    confirm_button.pack(side=tk.LEFT, padx=10)

    cancel_button = tk.Button(confirm_window, text="No", command=no)
    cancel_button.pack(side=tk.LEFT)

def submit_input():
    input_data = input_entry.get()
    # Perform any desired actions with the input data
    send_data(input_data)
    inputcorrection.destroy()

def inputcorrection():
    # Create the input correction window
    input_window = tk.Tk()

    # Set the window title
    input_window.title("Input coordinate shown by the camera")

    # Create an entry for the input string
    input_entry = tk.Entry(input_window)
    input_entry.pack()

    # Create a submit button
    submit_button = tk.Button(input_window, text="Submit", command=submit_input)
    submit_button.pack()

    # Run the input correction window
    input_window.mainloop()
    

def mode_1():
    def submit_degree():
        degree = degree_entry.get()
        send_data(degree)
        degree_window.destroy()
        koordinat=(0,0)
        Feedback_camera()
#         koreksi_manual()

    send_data("1")
    # Create a new window
    degree_window = tk.Toplevel(main_window)

    # Set the window title
    degree_window.title("Degree Input")

    # Create a label and an entry for degree input
    degree_label = tk.Label(degree_window, text="Enter a degree:")
    degree_label.pack()

    degree_entry = tk.Entry(degree_window)
    degree_entry.pack()

    # Create a button to show the inputted degree
    show_button = tk.Button(degree_window, text="Submit Degree", command=submit_degree)
    show_button.pack()

def mode_2():
    global koordinat
    def input_camera():
        # Global variables to store the clicked coordinates and flag
        clicked_coordinates = []
        clicked = False

        # Function to handle mouse events
        def click_event(event, x, y, flags, param):
            nonlocal clicked_coordinates, clicked, conversion_factor_x, conversion_factor_y

            if event == cv2.EVENT_LBUTTONDOWN:
                if not clicked:
                    # Clear the clicked coordinates and overlay image
                    clicked_coordinates.clear()
                    overlay.fill(0)

                    # Store the clicked coordinates
                    clicked_coordinates.append((x, y))
                    clicked = True

                    # Convert coordinates to fov_width x fov_height cm scale
                    converted_x = (frame_width - x) * conversion_factor_x
                    converted_y = y * conversion_factor_y
                    clicked_coordinates.append((converted_x, converted_y))

        # Create a video capture object
        cap = kamera2

        # Check if the camera is opened correctly
        if not cap.isOpened():
            raise IOError("Cannot open webcam")

        # Get the frame width and height
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Create an overlay image to draw grid lines
        overlay = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

        # Define the camera's field of view dimensions in centimeters
        fov_width_cm = 30
        fov_height_cm = 20

        # Calculate conversion factors
        conversion_factor_x = fov_width_cm / frame_width
        conversion_factor_y = fov_height_cm / frame_height

        # Draw grid lines on the overlay image
        grid_spacing_cm = 0.1  # Adjust this value to change the grid spacing in centimeters
        grid_spacing_px_x = int(grid_spacing_cm / conversion_factor_x)
        grid_spacing_px_y = int(grid_spacing_cm / conversion_factor_y)

        start_x = frame_width - grid_spacing_px_x
        while start_x >= 0:
            cv2.line(overlay, (start_x, 0), (start_x, frame_height), (255, 255, 255), 1)
            start_x -= grid_spacing_px_x

        for y in range(grid_spacing_px_y, frame_height, grid_spacing_px_y):
            cv2.line(overlay, (0, y), (frame_width, y), (255, 255, 255), 1)

        while True:
            # Read the current frame from the camera
            ret, frame = cap.read()

            # Copy the frame and overlay image for drawing
            output = frame.copy()
            overlay_copy = overlay.copy()

            # Add the overlay image on top of the frame
            output = cv2.addWeighted(output, 1, overlay_copy, 1, 0)

            # Display the output frame
            cv2.imshow("Camera Feed", output)

            # Set mouse handler for the window
            cv2.setMouseCallback("Camera Feed", click_event)

            # Break the loop if 'q' is pressed or a coordinate is clicked
            if cv2.waitKey(1) & 0xFF == ord("q") or clicked:
                break

        # Release the video capture object and close the windows
#         cap.release()
        cv2.destroyWindow('Camera Feed')
#         while(cap.isOpened()):
#             cap.release()
#             cv2.destroyAllWindows()


        # Return the clicked coordinates
        if clicked_coordinates:
            return clicked_coordinates[1]  # Return the converted coordinates
        else:
            return None





    def confirm_coordinate(coordinate):
        def confirm():
            # Run mode 2
            send_data("2")
            send_data(str(coordinate[0]) + "," + str(coordinate[1]) + ",0")
            confirm_window.destroy()
            Feedback_camera()

        def cancel():
            confirm_window.destroy()
            mode_2()

        # Create a new window for confirmation
        confirm_window = tk.Toplevel(main_window)
        confirm_window.title("Confirm Coordinate")

        # Show the coordinate
        coordinate_label = tk.Label(confirm_window, text="Selected Coordinate:")
        coordinate_label.pack()

        coordinate_value = tk.Label(confirm_window, text=f"({coordinate[0]}, {coordinate[1]})")
        coordinate_value.pack()

        # Create buttons for confirmation and cancellation
        confirm_button = tk.Button(confirm_window, text="Confirm", command=confirm)
        confirm_button.pack(side=tk.LEFT, padx=10)

        cancel_button = tk.Button(confirm_window, text="Cancel", command=cancel)
        cancel_button.pack(side=tk.LEFT)

    # Capture the coordinate
    coordinates = input_camera()
    coordinates = (round(coordinates[0]), round(coordinates[1]))
    koordinat=coordinates

    if coordinates:
        # Show the selected coordinate and ask for confirmation
        confirm_coordinate(coordinates)
    else:
        messagebox.showinfo("Coordinates", "No coordinates selected")

import cv2
import numpy as np
from ultralytics import YOLO

def draw_center_point(frame):
    # Mengambil dimensi frame
    height, width, _ = frame.shape

    # Menghitung titik tengah kamera
    center_x = width // 2
    center_y = height // 2

    # Menggambar titik biru di tengah kamera
    radius = 5
    color = (255, 0, 0)  # Blue color
    thickness = -1  # Filled circle
    cv2.circle(frame, (center_x, center_y), radius, color, thickness)

def Feedback_camera():
    last_coordinate = None  # Initialize the last coordinate
    def show_coord(frame, x1, y1, x2, y2, conversion_factor_x, conversion_factor_y):
        # Flip the x-coordinate
        flipped_x = frame.shape[1] - x2

        # Convert the coordinates to 20x20 cm scale
        converted_x = flipped_x * conversion_factor_x
        converted_y = y2 * conversion_factor_y

        # Display the converted coordinates on the frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        bottom_left = (int(x1), int(y2))
#         cv2.putText(frame, f"({converted_x:.2f}, {converted_y:.2f})", bottom_left, font, 0.6, (0, 0, 255), 2)

    # Load the YOLO model
    model = YOLO('../modelTA/hasil-training-colab/train5/best.pt')

    # Create a video capture object
    camera = kamera1
    img_counter = 0
    

    # Define the camera's field of view dimensions in centimeters
    fov_width_cm = 34
    fov_height_cm = 10

    while True:
        ret, frame = camera.read()

        if not ret:
            print("failed to grab frame")
            break
        cv2.imshow("test", frame)

        # Gambar titik biru di tengah kamera
        draw_center_point(frame)

        # Calculate conversion factors
        frame_width = frame.shape[1]
        frame_height = frame.shape[0]
        conversion_factor_x = fov_width_cm / frame_width
        conversion_factor_y = fov_height_cm / frame_height

        # Predict on the current frame
        outs = model.predict(frame)

        # Iterate over the detected bounding boxes and draw them on the frame
        for out in outs:
            boxes = out.boxes.xyxy.cpu().numpy()
            for box in boxes:
                x1, y1, x2, y2 = box
                show_coord(frame, x1, y1, x2, y2, conversion_factor_x, conversion_factor_y)
                flipped_x = frame.shape[1] - x2

                # Convert the coordinates to 20x20 cm scale
                converted_x = flipped_x * conversion_factor_x
                converted_y = y2 * conversion_factor_y
                last_coordinate = (converted_x, converted_y)
#                 last_coordinate = (x1, y1)  # Update the last coordinate

        # Display the frame with bounding boxes and converted coordinates
        cv2.imshow("test", frame)

        if cv2.waitKey(1) == 27:
            
            # Press ESC to exit
            if last_coordinate:
                x, y = last_coordinate
#                 converted_x = x * conversion_factor_x
#                 converted_y = y * conversion_factor_y
                print(f"Last Needle Coordinate (converted): ({converted_x:.2f}, {converted_y:.2f})")

                # Calculate the difference between the last coordinate and the camera's vertical midpoint
                camera_midpoint_y = (frame_height / 2)* conversion_factor_y
                camera_midpoint_x = (frame_width / 2)* conversion_factor_x
                difference_y = (camera_midpoint_y) - converted_y
                difference_x = (camera_midpoint_x) - (converted_x+6)
                print(f"center coordinate (converted): ({camera_midpoint_x:.2f}, {camera_midpoint_y:.2f})")

                print(f"Difference: {difference_x/10},{difference_y}")
#                 send data difference for pre inplane correction
                send_data(str(difference_x/10) + "," + str(difference_y))
                break
            else:
                print("No coordinates found")
                break
    # Release the camera and close the OpenCV window
#     camera.release()
    cv2.destroyWindow('test')
#     while(camera.isOpened()):
#         camera.release()
#         cv2.destroyAllWindows()
    
    koreksi_manual()

def koreksi_manual():
    
    def move_up():
        print("Moving up...")
        send_data("1")

    def move_down():
        print("Moving down...")
        send_data("2")

    def move_left():
        print("Moving left...")
        send_data("3")

    def move_right():
        print("Moving right...")
        send_data("4")

    def close_window():
        send_data("5")
        manual.destroy()
        try:
            print(koordinat)
            Display_penusukan(koordinat)
        except NameError:
            Display_penusukan((0,0))

        



    manual = tk.Tk()



    button_width = 10
    button_height = 5

    up_button = tk.Button(manual, text="↑", command=move_up, width=button_width, height=button_height)
    up_button.grid(row=1, column=1, pady=5)  # Adjusted row and added pady

    down_button = tk.Button(manual, text="↓", command=move_down, width=button_width, height=button_height)
    down_button.grid(row=2, column=1, pady=5)  # Adjusted row and added pady

    left_button = tk.Button(manual, text="←", command=move_left, width=button_width, height=button_height)
    left_button.grid(row=2, column=0, padx=5)  # Adjusted column and added padx

    right_button = tk.Button(manual, text="→", command=move_right, width=button_width, height=button_height)
    right_button.grid(row=2, column=2, padx=5)  # Adjusted column and added padx

    done_button = tk.Button(manual, text="Done", command=close_window, width=button_width, height=button_height)
    done_button.grid(row=3, column=1, pady=5)  # Adjusted row and added pady


    manual.mainloop()


from PIL import Image, ImageTk
def Display_penusukan(clicked_coordinates):
    # Create a video capture object
    cap = kamera2

    # Check if the camera is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    # Get the frame width and height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create an overlay image to draw grid lines and red dot
    overlay = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

    # Define the camera's field of view dimensions in centimeters
    fov_width_cm = 30
    fov_height_cm = 20

    # Calculate conversion factors
    conversion_factor_x = fov_width_cm / frame_width
    conversion_factor_y = fov_height_cm / frame_height

    # Draw grid lines on the overlay image
    grid_spacing_cm = 0.1  # Adjust this value to change the grid spacing in centimeters
    grid_spacing_px_x = int(grid_spacing_cm / conversion_factor_x)
    grid_spacing_px_y = int(grid_spacing_cm / conversion_factor_y)

    start_x = frame_width - grid_spacing_px_x
    while start_x >= 0:
        cv2.line(overlay, (start_x, 0), (start_x, frame_height), (255, 255, 255), 1)
        start_x -= grid_spacing_px_x

    for y in range(grid_spacing_px_y, frame_height, grid_spacing_px_y):
        cv2.line(overlay, (0, y), (frame_width, y), (255, 255, 255), 1)

    # Create a Tkinter window
    window = tk.Toplevel()
    window.title("Camera Feed")

    # Create a canvas to display the video feed
    canvas = tk.Canvas(window, width=frame_width, height=frame_height)
    canvas.pack()

    def update_frame():
        nonlocal cap, overlay, canvas, clicked_coordinates
        # Read the current frame from the camera
        ret, frame = cap.read()

        # Copy the frame and overlay image for drawing
        output = frame.copy()
        overlay_copy = overlay.copy()

        # Draw a red dot for the clicked coordinate, if available
        if clicked_coordinates:
            clicked_x = frame_width - int(clicked_coordinates[0] / conversion_factor_x)
            clicked_y = int(clicked_coordinates[1] / conversion_factor_y)
            cv2.circle(overlay_copy, (clicked_x, clicked_y), 5, (0, 0, 255), -1)

            # Add text displaying the coordinates under the drawn dot
            text = f"({clicked_coordinates[0]:.0f}, {clicked_coordinates[1]:.0f})"
#             cv2.putText(overlay_copy, text, (clicked_x - 20, clicked_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Add the overlay image on top of the frame
        output = cv2.addWeighted(output, 1, overlay_copy, 1, 0)

        # Convert the OpenCV frame to PIL format
        image = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(image)

        # Convert the PIL image to Tkinter format
        image_tk = ImageTk.PhotoImage(image_pil)

        # Update the canvas with the new image
        canvas.create_image(0, 0, anchor=tk.NW, image=image_tk)
        canvas.image = image_tk

        # Call this function again after 1ms
        window.after(1, update_frame)


    # Start updating the frame
    update_frame()

    # Run the Tkinter event loop
    window.mainloop()

def GerakZ():
    def submit_z():
        z = z_entry.get()
        send_data(z)
        z_window.destroy()
#         koreksi_manual()

    send_data("3")
    # Create a new window
    z_window = tk.Toplevel(main_window)

    # Set the window title
    z_window.title("Gerak Z")

    # Create a label and an entry for degree input
    degree_label = tk.Label(z_window, text="Masukkan Z dalam cm (-) untuk maju dan (+) untuk mundur:")
    degree_label.pack()

    z_entry = tk.Entry(z_window)
    z_entry.pack()

    # Create a button to show the inputted degree
    show_button = tk.Button(z_window, text="Submit", command=submit_z)
    show_button.pack()

def create_main_window():
    # Create the main window
    global main_window
    main_window = tk.Tk()

    # Set the window title
    main_window.title("Aplikasi Alat Uji Penusukan Jarum Spinal")

    # Create a label and buttons for mode selection
    mode_label = tk.Label(main_window, text="Aplikasi Alat Uji Penusukan Jarum Spinal")
    mode_label.pack()

    mode_label = tk.Label(main_window, text="Pilih Mode:")
    mode_label.pack()

    mode_1_button = tk.Button(main_window, text="Mode Input Sudut", command=mode_1)
    mode_1_button.pack()

    mode_2_button = tk.Button(main_window, text="Mode Input Koordinat", command=mode_2)
    mode_2_button.pack()
    
    Zaxis = tk.Button(main_window, text="Gerak Z", command=GerakZ)
    Zaxis.pack()


    feedback_button = tk.Button(main_window, text="Kamera Penusukan", command=Display_penusukan)
    feedback_button.pack()

    # Create a button to open the serial data window
    serial_button = tk.Button(main_window, text="Open Serial Data", command=create_serial_window)
    serial_button.pack()

    # Run the main event loop
    main_window.mainloop()


# Create the main window
create_main_window()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Legion-5-Izzul\anaconda3\envs\env2\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Legion-5-Izzul\AppData\Local\Temp\ipykernel_12544\1677837609.py", line 283, in mode_2
    coordinates = (round(coordinates[0]), round(coordinates[1]))
TypeError: 'NoneType' object is not subscriptable


untuk mereset port dan kamera

In [10]:
ser.setDTR(False)

# Wait for a short duration (e.g., 0.1 seconds)
time.sleep(0.1)

# Set DTR to True
ser.setDTR(True)

ser.close()
kamera1.release()
kamera2.release()